In [0]:
!pip install azure-kusto-data
!pip install azure-kusto-ingest

     |████████████████████████████████| 42 kB 618 kB/s 
     |████████████████████████████████| 133 kB 5.3 MB/s 
     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 130 kB 12.3 MB/s 
     |████████████████████████████████| 3.6 MB 21.9 MB/s 
     |████████████████████████████████| 178 kB 24.4 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 139 kB 8.4 MB/s 
     |████████████████████████████████| 346 kB 41.1 MB/s 
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 41 kB 756 kB/s 
     |████████████████████████████████| 151 kB 23.6 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

In [0]:
AAD_TENANT_ID = "<TENANT_ID>"            # from Active Directory
KUSTO_URI = "<KUSTO_URI>"                # from ADX overview page
KUSTO_INGEST_URI = "<KUSTO_INGEST_URI>"  # from ADX overview page
KUSTO_DATABASE = "<DATABASE_NAME>"       # from ADX overview page

In [0]:
KCSB_INGEST = KustoConnectionStringBuilder.with_aad_device_authentication(
KUSTO_INGEST_URI, AAD_TENANT_ID)
KCSB_DATA = KustoConnectionStringBuilder.with_aad_device_authentication(
KUSTO_URI, AAD_TENANT_ID)
DESTINATION_TABLE = "StormEvents"
DESTINATION_TABLE_COLUMN_MAPPING = "StormEvents_CSV_Mapping"

In [0]:
from azure.kusto.ingest import QueuedIngestClient, IngestionProperties, FileDescriptor, BlobDescriptor, ReportLevel, ReportMethod
from azure.kusto.data import DataFormat

CONTAINER = "samplefiles"
ACCOUNT_NAME = "kustosamples"
SAS_TOKEN = ""  # If relevant add SAS token
FILE_PATH = "StormEvents.csv"
FILE_SIZE = 64158321    # in bytes

BLOB_PATH = "https://" + ACCOUNT_NAME + ".blob.core.windows.net/" + \
    CONTAINER + "/" + FILE_PATH + SAS_TOKEN

In [0]:
KUSTO_CLIENT = KustoClient(KCSB_DATA)
CREATE_TABLE_COMMAND = ".create table StormEvents (StartTime: datetime, EndTime: datetime, EpisodeId: int, \
EventId: int, State: string, EventType: string, InjuriesDirect: int, InjuriesIndirect: int, DeathsDirect: \
int, DeathsIndirect: int, DamageProperty: int, DamageCrops: int, Source: string, BeginLocation: string, \
EndLocation: string, BeginLat: real, BeginLon: real, EndLat: real, EndLon: real, EpisodeNarrative: string, \
EventNarrative: string, StormSummary: dynamic)"
RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_TABLE_COMMAND)
dataframe_from_result_table(RESPONSE.primary_results[0])

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RFGBTBS7P to authenticate.


,TableName,Schema,DatabaseName,Folder,DocString
0,StormEvents,"{""Name"":""StormEvents"",""OrderedColumns"":[{""Name...",MyDatabase,None,None


In [0]:
CREATE_MAPPING_COMMAND = """.create table StormEvents ingestion csv mapping 'StormEvents_CSV_Mapping' '[{"Name":"StartTime","datatype":"datetime","Ordinal":0}, {"Name":"EndTime","datatype":"datetime","Ordinal":1},{"Name":"EpisodeId","datatype":"int","Ordinal":2},{"Name":"EventId","datatype":"int","Ordinal":3},{"Name":"State","datatype":"string","Ordinal":4},{"Name":"EventType","datatype":"string","Ordinal":5},{"Name":"InjuriesDirect","datatype":"int","Ordinal":6},{"Name":"InjuriesIndirect","datatype":"int","Ordinal":7},{"Name":"DeathsDirect","datatype":"int","Ordinal":8},{"Name":"DeathsIndirect","datatype":"int","Ordinal":9},{"Name":"DamageProperty","datatype":"int","Ordinal":10},{"Name":"DamageCrops","datatype":"int","Ordinal":11},{"Name":"Source","datatype":"string","Ordinal":12},{"Name":"BeginLocation","datatype":"string","Ordinal":13},{"Name":"EndLocation","datatype":"string","Ordinal":14},{"Name":"BeginLat","datatype":"real","Ordinal":16},{"Name":"BeginLon","datatype":"real","Ordinal":17},{"Name":"EndLat","datatype":"real","Ordinal":18},{"Name":"EndLon","datatype":"real","Ordinal":19},{"Name":"EpisodeNarrative","datatype":"string","Ordinal":20},{"Name":"EventNarrative","datatype":"string","Ordinal":21},{"Name":"StormSummary","datatype":"dynamic","Ordinal":22}]'"""

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_MAPPING_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,Name,Kind,Mapping,LastUpdatedOn,Database,Table
0,StormEvents_CSV_Mapping,Csv,"[{""Name"":""StartTime"",""DataType"":""datetime"",""Or...",2022-04-03 13:30:54.121358700+00:00,MyDatabase,StormEvents


In [0]:
INGESTION_CLIENT = QueuedIngestClient(KCSB_INGEST)

# All ingestion properties are documented here: https://docs.microsoft.com/azure/kusto/management/data-ingest#ingestion-properties
INGESTION_PROPERTIES = IngestionProperties(database=KUSTO_DATABASE, table=DESTINATION_TABLE, data_format=DataFormat.CSV,
                                           ingestion_mapping_reference=DESTINATION_TABLE_COLUMN_MAPPING, additional_properties={'ignoreFirstRecord': 'true'})
# FILE_SIZE is the raw size of the data in bytes
BLOB_DESCRIPTOR = BlobDescriptor(BLOB_PATH, FILE_SIZE)
INGESTION_CLIENT.ingest_from_blob(
    BLOB_DESCRIPTOR, ingestion_properties=INGESTION_PROPERTIES)

print('Done queuing up ingestion with Azure Data Explorer')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AH8UUVBUA to authenticate.
Done queuing up ingestion with Azure Data Explorer


In [0]:
QUERY = "StormEvents | count"

RESPONSE = KUSTO_CLIENT.execute_query(KUSTO_DATABASE, QUERY)

dataframe_from_result_table(RESPONSE.primary_results[0])

,Count
0,59066


In [0]:
QUERY = "StormEvents | sort by StartTime desc | take 10"

RESPONSE = KUSTO_CLIENT.execute_query(KUSTO_DATABASE, QUERY)

dataframe_from_result_table(RESPONSE.primary_results[0])

,StartTime,EndTime,EpisodeId,EventId,State,EventType,InjuriesDirect,InjuriesIndirect,DeathsDirect,DeathsIndirect,...,Source,BeginLocation,EndLocation,BeginLat,BeginLon,EndLat,EndLon,EpisodeNarrative,EventNarrative,StormSummary
0,2007-12-31 23:53:00+00:00,2007-12-31 23:53:00+00:00,12037,65839,CALIFORNIA,High Wind,0,0,0,0,...,Mesonet,,,<NA>,<NA>,<NA>,<NA>,North to northeast winds gusting to around 58 ...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
1,2007-12-31 23:53:00+00:00,2007-12-31 23:53:00+00:00,12037,65838,CALIFORNIA,High Wind,0,0,0,0,...,Mesonet,,,<NA>,<NA>,<NA>,<NA>,The Warm Springs RAWS sensor reported northerl...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
2,2007-12-31 22:30:00+00:00,2007-12-31 23:59:00+00:00,12950,71588,MICHIGAN,Winter Storm,0,0,0,0,...,Trained Spotter,,,<NA>,<NA>,<NA>,<NA>,This heavy snow event continued into the early...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
3,2007-12-31 22:30:00+00:00,2007-12-31 23:59:00+00:00,12950,71589,MICHIGAN,Winter Storm,0,0,0,0,...,Trained Spotter,,,<NA>,<NA>,<NA>,<NA>,This heavy snow event continued into the early...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
4,2007-12-31 22:30:00+00:00,2007-12-31 23:59:00+00:00,12950,71590,MICHIGAN,Winter Storm,0,0,0,0,...,Trained Spotter,,,<NA>,<NA>,<NA>,<NA>,This heavy snow event continued into the early...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
5,2007-12-31 20:30:00+00:00,2007-12-31 23:59:00+00:00,12950,71586,MICHIGAN,Winter Storm,0,0,0,0,...,Trained Spotter,,,<NA>,<NA>,<NA>,<NA>,This heavy snow event continued into the early...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
6,2007-12-31 20:30:00+00:00,2007-12-31 23:59:00+00:00,12950,71587,MICHIGAN,Winter Storm,0,0,0,0,...,Trained Spotter,,,<NA>,<NA>,<NA>,<NA>,This heavy snow event continued into the early...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
7,2007-12-31 19:00:00+00:00,2007-12-31 23:59:00+00:00,12994,71915,ALASKA,High Wind,0,0,0,0,...,ASOS,,,<NA>,<NA>,<NA>,<NA>,Sitka observed a measured gust of 48 MPH at 22...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
8,2007-12-31 19:00:00+00:00,2007-12-31 23:59:00+00:00,12994,71917,ALASKA,High Wind,0,0,0,0,...,AWOS,,,<NA>,<NA>,<NA>,<NA>,Craig reported very windy with higher gusts fr...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
9,2007-12-31 18:30:00+00:00,2007-12-31 23:59:00+00:00,13007,71979,CALIFORNIA,High Wind,0,0,0,0,...,Mesonet,,,<NA>,<NA>,<NA>,<NA>,A 72 mph wind gusts was measured by the Tonner...,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
